In [0]:
df = spark.read.format("json").option("header", "True").load("dbfs:/FileStore/tables/ecommerce_comments.json")
df = df.select('cinsiyet', 'comments', 'kategori', 'marka', 'page_number', 'product_name', 'stars')
df.show()


+--------+--------------------+--------+-----------+-----------+--------------------+-------+
|cinsiyet|            comments|kategori|      marka|page_number|        product_name|  stars|
+--------+--------------------+--------+-----------+-----------+--------------------+-------+
|   erkek|['Gayet güzel uma...|   sapka|calvinklein|          1|   Calvin KleinŞapka|['4.3']|
|   erkek|                  []|   sapka|calvinklein|          2|Calvin KleinSiyah...|['5.0']|
|   erkek|['Güzel ürün', 'Ü...|   sapka|calvinklein|          3|Calvin KleinMavi ...|['4.5']|
|   erkek|                  []|   sapka|calvinklein|          4|   Calvin KleinŞapka|     []|
|   erkek|['Baba beğenmiş !...|   sapka|calvinklein|          5|   Calvin KleinŞapka|['4.5']|
|   erkek|                  []|   sapka|calvinklein|          6|Calvin KleinSPORT...|     []|
|   erkek|                  []|   sapka|calvinklein|          7|Calvin KleinSiyah...|     []|
|   erkek|['Eşim için aldım...|   sapka|calvinklein|        

In [0]:
df.select("comments").show(1,truncate=False) #ilk gözlem

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|comments                                                                                                                                                                                                                                                                                                     |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|['Gayet güzel umarım orijinaldir', "In the photo there is a textile clasp, it arrived w

In [0]:
total_rows = df.count()
total_columns = len(df.columns)

print("Verideki Toplam satır sayısı:", total_rows)
print("Verideki Toplam sütun sayısı:", total_columns)

Verideki Toplam satır sayısı: 2739
Verideki Toplam sütun sayısı: 7


In [0]:
from pyspark.sql.functions import col
for column in df.columns:
    null_count = df.where(col(column).isNull()).count()
    print(f"{column}: {null_count} null values")

# Verideki [] olan gözlemleri kayıp değer kabul ederek işlemleri buna göre devam ettireceğiz.


cinsiyet: 0 null values
comments: 0 null values
kategori: 0 null values
marka: 0 null values
page_number: 0 null values
product_name: 0 null values
stars: 0 null values


In [0]:
# "comments" değişkeninde "[]" (Kayıp gözlem olarak kabul edildi) eşit olmayan gözlemleri filtreleme
df = df.filter(df["comments"] != "[]")

# Filtrelenmiş veriyi gösterme
df.show()


+--------+--------------------+--------+-----------+-----------+--------------------+-------+
|cinsiyet|            comments|kategori|      marka|page_number|        product_name|  stars|
+--------+--------------------+--------+-----------+-----------+--------------------+-------+
|   erkek|['Gayet güzel uma...|   sapka|calvinklein|          1|   Calvin KleinŞapka|['4.3']|
|   erkek|['Güzel ürün', 'Ü...|   sapka|calvinklein|          3|Calvin KleinMavi ...|['4.5']|
|   erkek|['Baba beğenmiş !...|   sapka|calvinklein|          5|   Calvin KleinŞapka|['4.5']|
|   erkek|['Eşim için aldım...|   sapka|calvinklein|          8|Calvin KleinCK Lo...|['4.7']|
|   erkek|['Çok beğendim 😊...|   sapka|calvinklein|         12|   Calvin KleinŞapka|['5.0']|
|   erkek|['yanlış ürün yol...|   sapka|calvinklein|         16|Calvin KleinLogol...|['4.2']|
|   erkek|['Kumaşı kalın. Y...|   sapka|calvinklein|         17|Calvin KleinErkek...|['4.3']|
|   erkek|['Gayet güzel uma...|   sapka|calvinklein|         

In [0]:
# "page_number" sütununu DataFrame'den çıkarma
df = df.drop("page_number")

# Güncellenmiş DataFrame'i gösterme
df.show()


+--------+--------------------+--------+-----------+--------------------+-------+
|cinsiyet|            comments|kategori|      marka|        product_name|  stars|
+--------+--------------------+--------+-----------+--------------------+-------+
|   erkek|['Gayet güzel uma...|   sapka|calvinklein|   Calvin KleinŞapka|['4.3']|
|   erkek|['Güzel ürün', 'Ü...|   sapka|calvinklein|Calvin KleinMavi ...|['4.5']|
|   erkek|['Baba beğenmiş !...|   sapka|calvinklein|   Calvin KleinŞapka|['4.5']|
|   erkek|['Eşim için aldım...|   sapka|calvinklein|Calvin KleinCK Lo...|['4.7']|
|   erkek|['Çok beğendim 😊...|   sapka|calvinklein|   Calvin KleinŞapka|['5.0']|
|   erkek|['yanlış ürün yol...|   sapka|calvinklein|Calvin KleinLogol...|['4.2']|
|   erkek|['Kumaşı kalın. Y...|   sapka|calvinklein|Calvin KleinErkek...|['4.3']|
|   erkek|['Gayet güzel uma...|   sapka|calvinklein|   Calvin KleinŞapka|['4.3']|
|   erkek|['Güzel ürün', 'Ü...|   sapka|calvinklein|Calvin KleinMavi ...|['4.5']|
|   erkek|['Baba 

In [0]:
from pyspark.sql.functions import regexp_replace

# "stars" sütunundaki string değerlerin içerisindeki "[" ve "]" karakterlerini kaldırma
df = df.withColumn("stars", regexp_replace("stars", "\[|\]", ""))

# Güncellenmiş DataFrame'i gösterme
df.show()


+--------+--------------------+--------+-----------+--------------------+-----+
|cinsiyet|            comments|kategori|      marka|        product_name|stars|
+--------+--------------------+--------+-----------+--------------------+-----+
|   erkek|['Gayet güzel uma...|   sapka|calvinklein|   Calvin KleinŞapka|'4.3'|
|   erkek|['Güzel ürün', 'Ü...|   sapka|calvinklein|Calvin KleinMavi ...|'4.5'|
|   erkek|['Baba beğenmiş !...|   sapka|calvinklein|   Calvin KleinŞapka|'4.5'|
|   erkek|['Eşim için aldım...|   sapka|calvinklein|Calvin KleinCK Lo...|'4.7'|
|   erkek|['Çok beğendim 😊...|   sapka|calvinklein|   Calvin KleinŞapka|'5.0'|
|   erkek|['yanlış ürün yol...|   sapka|calvinklein|Calvin KleinLogol...|'4.2'|
|   erkek|['Kumaşı kalın. Y...|   sapka|calvinklein|Calvin KleinErkek...|'4.3'|
|   erkek|['Gayet güzel uma...|   sapka|calvinklein|   Calvin KleinŞapka|'4.3'|
|   erkek|['Güzel ürün', 'Ü...|   sapka|calvinklein|Calvin KleinMavi ...|'4.5'|
|   erkek|['Baba beğenmiş !...|   sapka|c

In [0]:
df = df.withColumn("stars", regexp_replace("stars", "'", ""))

# Güncellenmiş DataFrame'i gösterme
df.show()

+--------+--------------------+--------+-----------+--------------------+-----+
|cinsiyet|            comments|kategori|      marka|        product_name|stars|
+--------+--------------------+--------+-----------+--------------------+-----+
|   erkek|['Gayet güzel uma...|   sapka|calvinklein|   Calvin KleinŞapka|  4.3|
|   erkek|['Güzel ürün', 'Ü...|   sapka|calvinklein|Calvin KleinMavi ...|  4.5|
|   erkek|['Baba beğenmiş !...|   sapka|calvinklein|   Calvin KleinŞapka|  4.5|
|   erkek|['Eşim için aldım...|   sapka|calvinklein|Calvin KleinCK Lo...|  4.7|
|   erkek|['Çok beğendim 😊...|   sapka|calvinklein|   Calvin KleinŞapka|  5.0|
|   erkek|['yanlış ürün yol...|   sapka|calvinklein|Calvin KleinLogol...|  4.2|
|   erkek|['Kumaşı kalın. Y...|   sapka|calvinklein|Calvin KleinErkek...|  4.3|
|   erkek|['Gayet güzel uma...|   sapka|calvinklein|   Calvin KleinŞapka|  4.3|
|   erkek|['Güzel ürün', 'Ü...|   sapka|calvinklein|Calvin KleinMavi ...|  4.5|
|   erkek|['Baba beğenmiş !...|   sapka|c

In [0]:
from pyspark.sql.functions import col

# 'stars' sütununu float'a dönüştürme
df = df.withColumn("stars", col("stars").cast("float"))

# Güncellenmiş DataFrame'i gösterme
df.show()

+--------+--------------------+--------+-----------+--------------------+-----+
|cinsiyet|            comments|kategori|      marka|        product_name|stars|
+--------+--------------------+--------+-----------+--------------------+-----+
|   erkek|['Gayet güzel uma...|   sapka|calvinklein|   Calvin KleinŞapka|  4.3|
|   erkek|['Güzel ürün', 'Ü...|   sapka|calvinklein|Calvin KleinMavi ...|  4.5|
|   erkek|['Baba beğenmiş !...|   sapka|calvinklein|   Calvin KleinŞapka|  4.5|
|   erkek|['Eşim için aldım...|   sapka|calvinklein|Calvin KleinCK Lo...|  4.7|
|   erkek|['Çok beğendim 😊...|   sapka|calvinklein|   Calvin KleinŞapka|  5.0|
|   erkek|['yanlış ürün yol...|   sapka|calvinklein|Calvin KleinLogol...|  4.2|
|   erkek|['Kumaşı kalın. Y...|   sapka|calvinklein|Calvin KleinErkek...|  4.3|
|   erkek|['Gayet güzel uma...|   sapka|calvinklein|   Calvin KleinŞapka|  4.3|
|   erkek|['Güzel ürün', 'Ü...|   sapka|calvinklein|Calvin KleinMavi ...|  4.5|
|   erkek|['Baba beğenmiş !...|   sapka|c

In [0]:
max_stars = df.select("stars").agg({"stars": "max"}).collect()[0][0]
min_stars = df.select("stars").agg({"stars": "min"}).collect()[0][0]

print("Max stars:", max_stars)
print("Min stars:", min_stars)
# 0-3 arası negatif yorumlar , 3 nötr yorumlar , 3-5 arası pozitif yorumlar

Max stars: 5.0
Min stars: 0.0


In [0]:
from pyspark.sql.functions import when
df = df.withColumn("sentiment", when(df["stars"] < 3, "negative").when(df["stars"] > 3, "positive").otherwise("neutral"))
# 0-3 arası negatif yorumlar , 3 nötr yorumlar , 3-5 arası pozitif yorumlar şekilde sentiment adı altında yeni değişken oluşturuldu

In [0]:
df.show()

+--------+--------------------+--------+-----------+--------------------+-----+---------+
|cinsiyet|            comments|kategori|      marka|        product_name|stars|sentiment|
+--------+--------------------+--------+-----------+--------------------+-----+---------+
|   erkek|['Gayet güzel uma...|   sapka|calvinklein|   Calvin KleinŞapka|  4.3| positive|
|   erkek|['Güzel ürün', 'Ü...|   sapka|calvinklein|Calvin KleinMavi ...|  4.5| positive|
|   erkek|['Baba beğenmiş !...|   sapka|calvinklein|   Calvin KleinŞapka|  4.5| positive|
|   erkek|['Eşim için aldım...|   sapka|calvinklein|Calvin KleinCK Lo...|  4.7| positive|
|   erkek|['Çok beğendim 😊...|   sapka|calvinklein|   Calvin KleinŞapka|  5.0| positive|
|   erkek|['yanlış ürün yol...|   sapka|calvinklein|Calvin KleinLogol...|  4.2| positive|
|   erkek|['Kumaşı kalın. Y...|   sapka|calvinklein|Calvin KleinErkek...|  4.3| positive|
|   erkek|['Gayet güzel uma...|   sapka|calvinklein|   Calvin KleinŞapka|  4.3| positive|
|   erkek|[

In [0]:

df.filter(
    (col("stars")<3)
    ).show() # kötü yorumlar


+--------+--------------------+---------------+----------+--------------------+-----+---------+
|cinsiyet|            comments|       kategori|     marka|        product_name|stars|sentiment|
+--------+--------------------+---------------+----------+--------------------+-----+---------+
|   kadin|['gayet güzel oğl...|    sirtcantasi|    hummel|HUMMEL Siyah Sırt...|  0.0| negative|
|   kadin|['Güzel beğendim ...|    sirtcantasi|    hummel|HUMMEL Backpack G...|  2.7| negative|
|   kadin|['İkinci defadır ...|    sirtcantasi|    hummel|HUMMEL Kırmızı Sı...|  2.0| negative|
|   kadin|['pox kimdi, alma...|    sirtcantasi|lumberjack|Lumberjack Busine...|  2.0| negative|
|   kadin|['pox kimdi, alma...|    sirtcantasi|lumberjack|Lumberjack Busine...|  2.0| negative|
|   kadin|['pox kimdi, alma...|    sirtcantasi|lumberjack|Lumberjack Busine...|  2.0| negative|
|   kadin|['pox kimdi, alma...|    sirtcantasi|lumberjack|Lumberjack Busine...|  2.0| negative|
|   kadin|['pox kimdi, alma...|    sirtc

In [0]:
df.filter(
    (col("sentiment") == "negative") &
    (col("marka")=="lumberjack")
    ).show()

#Lumberjack Markası kötü yorumlar

+--------+--------------------+-----------+----------+--------------------+-----+---------+
|cinsiyet|            comments|   kategori|     marka|        product_name|stars|sentiment|
+--------+--------------------+-----------+----------+--------------------+-----+---------+
|   kadin|['pox kimdi, alma...|sirtcantasi|lumberjack|Lumberjack Busine...|  2.0| negative|
|   kadin|['pox kimdi, alma...|sirtcantasi|lumberjack|Lumberjack Busine...|  2.0| negative|
|   kadin|['pox kimdi, alma...|sirtcantasi|lumberjack|Lumberjack Busine...|  2.0| negative|
|   kadin|['pox kimdi, alma...|sirtcantasi|lumberjack|Lumberjack Busine...|  2.0| negative|
|   kadin|['pox kimdi, alma...|sirtcantasi|lumberjack|Lumberjack Busine...|  2.0| negative|
|   kadin|['pox kimdi, alma...|sirtcantasi|lumberjack|Lumberjack Busine...|  2.0| negative|
+--------+--------------------+-----------+----------+--------------------+-----+---------+



In [0]:
# Yinelenen satırları kaldırma
df = df.dropDuplicates()

# Güncellenmiş DataFrame'i gösterme
df.show()


+--------+--------------------+--------+-----------+--------------------+-----+---------+
|cinsiyet|            comments|kategori|      marka|        product_name|stars|sentiment|
+--------+--------------------+--------+-----------+--------------------+-----+---------+
|   erkek|['Arkadaşıma aldı...|   sapka|       puma|   PumaMetal Cat Cap|  4.8| positive|
|   erkek|['Eşime aldım çok...|   sapka|       puma|PumaEss Running S...|  4.7| positive|
|   erkek|['kaliteli şık bi...|   sapka|       puma|PumaMetal Cat Cap...|  5.0| positive|
|   erkek|['Eşim için aldım...|   sapka|calvinklein|Calvin KleinCK Lo...|  4.7| positive|
|   erkek|['Baba beğenmiş !...|   sapka|calvinklein|   Calvin KleinŞapka|  4.5| positive|
|   erkek|['Çok beğendim 😊...|   sapka|calvinklein|   Calvin KleinŞapka|  5.0| positive|
|   erkek|['Güzel, sadece ç...|   sapka|       puma|PumaPembe Unisex ...|  5.0| positive|
|   erkek|['Güzel ürün', 'Ü...|   sapka|calvinklein|Calvin KleinMavi ...|  4.5| positive|
|   erkek|[

In [0]:
df.filter(
    (col("sentiment") == "negative") &
    (col("marka")=="lumberjack")
    ).show()

+--------+--------------------+-----------+----------+--------------------+-----+---------+
|cinsiyet|            comments|   kategori|     marka|        product_name|stars|sentiment|
+--------+--------------------+-----------+----------+--------------------+-----+---------+
|   kadin|['pox kimdi, alma...|sirtcantasi|lumberjack|Lumberjack Busine...|  2.0| negative|
+--------+--------------------+-----------+----------+--------------------+-----+---------+



In [0]:
df.filter(
    (col("sentiment") == "negative") &
    (col("marka")=="lumberjack")
    ).select("comments").show(truncate=False) #lumberjack markasındaki olumsuz yorum

+--------------------------------------------------------------------------------------------------------------------------------------------+
|comments                                                                                                                                    |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|['pox kimdi, almağa dəyməz, zamokun içinə sıçıb göndəriblər, ürünüvüzü sikim, zamokda plasmasdan iki günə qırılacağ, pulumu doğun qəhbələr']|
+--------------------------------------------------------------------------------------------------------------------------------------------+



In [0]:
from pyspark.sql.functions import desc

grouped_df = df.groupBy("marka").agg({"stars": "avg", "marka": "count"}) \
    .withColumnRenamed("count(marka)", "Markadaki_Yorum_Sayisi") \
    .withColumnRenamed("avg(stars)", "stars_rate")

sorted_df = grouped_df.orderBy(desc("stars_rate"))

sorted_df.show()

# Markaların kalitesi aldığı yorumlara göre sıralandı

+-----------+----------------------+------------------+
|      marka|Markadaki_Yorum_Sayisi|        stars_rate|
+-----------+----------------------+------------------+
|underarmour|                    11| 4.845454562794078|
|       puma|                    14| 4.814285687037876|
|    polaris|                    13| 4.761538468874418|
|      guess|                    12| 4.708333373069763|
|      mango|                    21|4.6142857528868175|
| lumberjack|                    18|4.5277778307596845|
|calvinklein|                     7|               4.5|
|     adidas|                    22| 4.495454506440596|
|   ninewest|                   158|  4.48924052262608|
|     hummel|                   149|4.4543624148272825|
|       nike|                    20| 4.400000035762787|
|     reebok|                    24| 4.329166640837987|
|       inci|                    23| 4.273913072503132|
|      bambi|                    18| 4.266666637526618|
|     marjin|                    24| 4.208333323

In [0]:
from pyspark.sql.functions import desc

grouped_df2 = df.groupBy("kategori").agg({"stars": "avg", "kategori": "count"}) \
    .withColumnRenamed("count(kategori)", "Kategorideki_Yorum_Sayisi") \
    .withColumnRenamed("avg(stars)", "stars_rate")

sorted_df2 = grouped_df2.orderBy(desc("stars_rate"))

sorted_df2.show()

# ürün kategorileri aldığı yorumlara göre sıralandı


+---------------+-------------------------+------------------+
|       kategori|Kategorideki_Yorum_Sayisi|        stars_rate|
+---------------+-------------------------+------------------+
|          sapka|                       32|4.7562499940395355|
|     kolcantasi|                       47| 4.572340452924688|
|    sirtcantasi|                      188|  4.46861702330569|
|   sporayakkabi|                       66| 4.406060594500917|
|  babetayakkabi|                       55| 4.358181806044145|
|topukluayakkabi|                      226| 4.353982317764147|
+---------------+-------------------------+------------------+



In [0]:
from pyspark.sql.functions import col

# data 'marka' sütununa göre gruplandı ve her bir markanın yorum sayısı hesaplandı
brand_counts = df.groupBy("kategori").count()

brand_counts.show()


+---------------+-----+
|       kategori|count|
+---------------+-----+
|    sirtcantasi|  188|
|          sapka|   32|
|     kolcantasi|   47|
|topukluayakkabi|  226|
|   sporayakkabi|   66|
|  babetayakkabi|   55|
+---------------+-----+



In [0]:
df.createOrReplaceTempView("df")
spark.sql("SELECT marka, COUNT(*) AS Yorum_Sayisi FROM df WHERE kategori = 'sirtcantasi' GROUP BY marka").show()


+----------+------------+
|     marka|Yorum_Sayisi|
+----------+------------+
|lumberjack|          18|
|   derimod|          21|
|    hummel|         149|
+----------+------------+



In [0]:
spark.sql("""
    SELECT marka, COUNT(*) AS Yorum_Sayisi, AVG(stars) AS stars_rate
    FROM df
    WHERE kategori = 'sirtcantasi'
    GROUP BY marka
    ORDER BY AVG(stars) DESC
""").show() # sirtcantasi markaları yildiz(memnuniyet) oranlari

+----------+------------+------------------+
|     marka|Yorum_Sayisi|        stars_rate|
+----------+------------+------------------+
|lumberjack|          18|4.5277778307596845|
|   derimod|          21| 4.519047600882394|
|    hummel|         149|4.4543624148272825|
+----------+------------+------------------+



In [0]:
spark.sql("select marka, count(*) as Yorum_Sayisi, AVG(stars) as stars_rate\
    FROM df where kategori = 'kolcantasi' group by marka order by avg(stars) desc").show() #kolcantasi markalari yildiz(memnuniyet) oranlari

+--------+------------+------------------+
|   marka|Yorum_Sayisi|        stars_rate|
+--------+------------+------------------+
|   guess|          12| 4.708333373069763|
|   mango|          21|4.6142857528868175|
|ninewest|          14| 4.392857142857143|
+--------+------------+------------------+



In [0]:
spark.sql("select marka, count(*) as Yorum_Sayisi, AVG(stars) as stars_rate\
    FROM df where kategori = 'topukluayakkabi' group by marka order by avg(stars) desc").show() #topukluayakkabi markalari yildiz(memnuniyet) oranlari

+--------+------------+-----------------+
|   marka|Yorum_Sayisi|       stars_rate|
+--------+------------+-----------------+
|ninewest|         144|4.498611128992504|
|    inci|          23|4.273913072503132|
| derimod|          59|4.032203399528892|
+--------+------------+-----------------+



In [0]:
spark.sql("select marka, count(*) as Yorum_Sayisi, AVG(stars) as stars_rate\
    FROM df where kategori = 'babetayakkabi' group by marka order by avg(stars) desc").show() #babetayakkabi markalari yildiz(memnuniyet) oranlari

+-------+------------+-----------------+
|  marka|Yorum_Sayisi|       stars_rate|
+-------+------------+-----------------+
|polaris|          13|4.761538468874418|
|  bambi|          18|4.266666637526618|
| marjin|          24|4.208333323399226|
+-------+------------+-----------------+



In [0]:
spark.sql("select marka, count(*) as Yorum_Sayisi, AVG(stars) as stars_rate\
    FROM df where kategori = 'sporayakkabi' group by marka order by avg(stars) desc").show() #sporayakkabi markalari yildiz(memnuniyet) oranlari

+------+------------+-----------------+
| marka|Yorum_Sayisi|       stars_rate|
+------+------------+-----------------+
|adidas|          22|4.495454506440596|
|  nike|          20|4.400000035762787|
|reebok|          24|4.329166640837987|
+------+------------+-----------------+



In [0]:
spark.sql("select marka, count(*) as Yorum_Sayisi, AVG(stars) as stars_rate\
    FROM df where kategori = 'sapka' group by marka order by avg(stars) desc").show() #sapka markalari yildiz(memnuniyet) oranlari

+-----------+------------+-----------------+
|      marka|Yorum_Sayisi|       stars_rate|
+-----------+------------+-----------------+
|underarmour|          11|4.845454562794078|
|       puma|          14|4.814285687037876|
|calvinklein|           7|              4.5|
+-----------+------------+-----------------+



In [0]:
total_rows = df.count()
total_columns = len(df.columns)

print("Verideki Toplam satır sayısı:", total_rows)
print("Verideki Toplam sütun sayısı:", total_columns)

Verideki Toplam satır sayısı: 614
Verideki Toplam sütun sayısı: 7


In [0]:
df.write.format("jdbc")\
    .option("url", "jdbc:postgresql://34.122.87.14:5432/postgres") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "ecommerce_comments") \
    .option("user", "postgres").option("password", "Lemtasiq7.") \
    .mode("append").save()